In [9]:
from mnist_loader import load_data, load_data_wrapper
import numpy as np
import matplotlib.pyplot as plt

Rozpakowuję dane

In [10]:
data = list(load_data_wrapper()[0])
data_2 = list(load_data_wrapper()[2])
train_data = (data[0],data[1])
test_data = (data_2[0],data_2[1])

FileNotFoundError: [Errno 2] No such file or directory: 'Data/mnist.pkl.gz'

Funkcja aktywacji sigmoid i jej pochodna

In [3]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

Klasa warstwa:

Metoda trigger - wartość po przemnożeniu wag z wyzwoleniem z poprzedniej warstwy

Atrybut weights - wektor wag danej warstwy

In [4]:
class Layer:
    def __init__(self, entry_size, size):
        self.entry = entry_size
        self.size = size
        weights = np.random.normal(size = (self.entry + 1)*self.size,scale = 1/(2*np.sqrt(self.entry)) )
        self.weights = np.reshape(weights,(self.size, self.entry + 1))
    
    def trigger(self, entry, function = sigmoid):
        entry = np.vstack((entry, -1))
        self.triggered = self.weights@entry
        return function(self.triggered)

Klasa sieć:

Metoda feedforward - aktywacja sieci, wyliczenie wyjścia

Metoda backprop - algorytm propagacji wsteczenj, uaktualnienie wag

Metoda fit - uczenie sieci, wrzucamy listę kolejnych wejść i wyjść, stosujemy online learning

Metoda predict - predykcja na podstawie wejście, gdy sieć jest już nauczona

Atrybut network - lista kolejnych warst

Atrybut entry - wartości wejściowe (macierz 784 pikseli)

Atrybut output - wartości wyjściowe z ostatniej wykonanej aktywacji

In [5]:
class Dense:
    def __init__(self, shape):
        self.network = []
        for i in range(len(shape)-1):
            self.network.append(Layer(shape[i],shape[i+1]))

    def feedforward(self,entry):
        self.entry = entry
        layer_triggered = self.network[0].trigger(self.entry)
        for i in self.network[1:]:
            layer_triggered = i.trigger(layer_triggered)
        self.output = layer_triggered
        return self.output
    
    def backprop(self, t,c=0.05):
        self.delta = self.delta + np.sum(((self.output - t).T)**2)
        delta = (self.output - t)*sigmoid_prime(self.network[-1].triggered)
        delta = delta.reshape((self.network[-1].size,1))
        z = self.network[-2].triggered
        z = np.vstack((z,-1)).reshape((1,self.network[-2].size + 1))
        prev_weights = self.network[-1].weights
        prev_delta = delta
        self.network[-1].weights = self.network[-1].weights - c*(delta@z)  
        for i in range(len(self.network) - 1):
            delta = (prev_delta.T@prev_weights)[0][:-1]*sigmoid_prime(self.network[-2-i].triggered).T
            delta = delta.T
            if (i + 2 == len(self.network)):
                prev_triggered = self.entry
                prev_triggered = np.vstack((prev_triggered,-1))
            else:
                prev_triggered = self.network[-3-i].triggered
                prev_triggered = np.vstack((prev_triggered,-1))
                prev_weights = self.network[-2-i].weights
                prev_delta = delta
            self.network[-2-i].weights = self.network[-2-i].weights - c*(delta@prev_triggered.T)
            
        
    def fit(self, data, test_data, epoch_num = 1, learning_rate = 0.05):
        input_data = data[0]
        output_data = data[1]
        n = len(output_data)
        for j in range(epoch_num):
            self.delta = 0
            for i in range(n):
                self.feedforward(input_data[i])
                self.backprop(output_data[i], learning_rate)
            y = 0
            for i in range(len(test_data[0])):
                y += (x.predict(test_data[0][i]) == test_data[1][i])
            print("Epoch", j+1,": Loss = ", self.delta, ", Accuracy = ", y/len(test_data[0]))
            
    def predict(self, data):
        y = self.feedforward(data)
        return np.argmax(y)

Przykładowa architektura sieci - 3 warstwy, (784, 30, 10) neuronów

Metodą fit uczymy sieć, wykorzustując zbiór treningowy

In [6]:
x = Dense([784,80,10])
x.fit(train_data,test_data, epoch_num = 30, learning_rate = 0.05)

Epoch 1 : Loss =  11101.29877412701 , Accuracy =  0.9248
Epoch 2 : Loss =  6317.906466566946 , Accuracy =  0.9385
Epoch 3 : Loss =  5343.207328608311 , Accuracy =  0.9448
Epoch 4 : Loss =  4752.881375659912 , Accuracy =  0.9483
Epoch 5 : Loss =  4335.88422726678 , Accuracy =  0.9515
Epoch 6 : Loss =  4017.571009531392 , Accuracy =  0.9527
Epoch 7 : Loss =  3760.403656012428 , Accuracy =  0.9553
Epoch 8 : Loss =  3544.6067100913433 , Accuracy =  0.9568
Epoch 9 : Loss =  3361.9464109456467 , Accuracy =  0.9577
Epoch 10 : Loss =  3207.0008355954933 , Accuracy =  0.9584
Epoch 11 : Loss =  3070.550313624855 , Accuracy =  0.9579
Epoch 12 : Loss =  2947.5643150879523 , Accuracy =  0.9579
Epoch 13 : Loss =  2836.2753654591243 , Accuracy =  0.9589
Epoch 14 : Loss =  2735.1400658286548 , Accuracy =  0.9599
Epoch 15 : Loss =  2641.665103796738 , Accuracy =  0.9604
Epoch 16 : Loss =  2555.0136520522815 , Accuracy =  0.961
Epoch 17 : Loss =  2475.1526442608233 , Accuracy =  0.961
Epoch 18 : Loss = 

Wyliczamy skuteczność modelu na podstawie zbioru testowego

In [7]:
y = 0
for i in range(len(test_data[0])):
    y += (x.predict(test_data[0][i]) == test_data[1][i])
y/10000

0.963